In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.metrics import f1_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob
print(os.listdir("../input"))


In [ ]:
np.random.seed(42)

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
## set the data folder
data_folder = Path("../input")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
cd /kaggle/input/he_challenge_data/data

In [ ]:
ls

In [ ]:
train=pd.read_csv("trainfinal.csv")
test=pd.read_csv("testfinal.csv")
train.head(), test.head()

In [ ]:
del test['image_id']

In [ ]:
train['image_id']= train['image_id'].astype(str) + '.jpg'
train.head(5)

In [ ]:
test['image_id']= test['image_id'].astype(str) + '.jpg'
test.head(5)

In [ ]:
submission = pd.DataFrame({'image_id': train['image_id'], 'category': train['label']})
submission.head(10)

In [ ]:
#submission.to_csv('trainfinal.csv', index=False)
train.to_csv("../input/HE_Challenge_data/data/trainfinal.csv", index= False)

In [ ]:
path = "/kaggle/input/he_challenge_data/data"

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

In [ ]:
!pip install https://github.com/fastai/fastai/archive/1.0.49.zip

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
!pip install fastai

In [ ]:
cd /kaggle/input/he_challenge_data/data

In [ ]:
sys.setrecursionlimit(50000)


In [ ]:
data = ImageDataBunch.from_csv(path=path, folder='train', csv_labels='trainfinal.csv', test='test', ds_tfms=get_transforms(), size=400, bs=8).normalize(imagenet_stats)

In [ ]:

get_transforms()

In [ ]:
data.show_batch()

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
!pip install pretrainedmodels
from torchvision.models import *
import pretrainedmodels

from fastai import *
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import fastai

from utils import *
import sys
import torch
fastai.__version__

In [ ]:
learn = cnn_learner(data, models.densenet161, metrics=[error_rate, accuracy], model_dir="/tmp/model/")


In [ ]:
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn.fit_one_cycle(2)

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-3))

In [ ]:
a,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
labelled_preds = []
for pred in a:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'category': labelled_preds,
    })
submission.to_csv('get.csv',index=False)

In [ ]:
labelled_preds

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=10)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1');

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.load('stage-1');
learn.unfreeze()
learn.fit_one_cycle(2)

In [ ]:
learn50 = create_cnn(data, models.resnet50, metrics=error_rate, model_dir="/tmp/model/")

In [ ]:
learn50.lr_find()
learn50.recorder.plot()

In [ ]:
learn50.fit_one_cycle(8)

In [ ]:
learn50.save('stage-1-50')

In [ ]:
learn50.load('stage-1-50')
learn50.unfreeze()
learn50.fit_one_cycle(5, max_lr=slice(1e-4,1e-2))

In [ ]:
log_preds, test_labels = learn50.get_preds(ds_type=DatasetType.Test)

In [ ]:
preds = np.argmax(log_preds, 1)
preds

In [ ]:
import numpy as np
import pandas as pd
a = np.array(preds)
print(a)
a=a+1
print(a.max)
print(data.test_ds.x[0])

In [ ]:
df = pd.DataFrame({'image_id':test['image_id'], 'label':predictions}, columns=['image', 'label'])
df.to_csv('c.csv', index=False)

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "c.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(filename = 'c.csv')

In [ ]:
sizes = [32, 64, 128, 224]

****2nd**  **Approch************

In [ ]:
path = "/kaggle/input/he_challenge_data/data"

In [ ]:
def get_data(sz, bs):
  data = ImageDataBunch.from_csv(path=path, folder='train', csv_labels='trainfinal.csv', test='test', ds_tfms=get_transforms(), size=sz, bs=bs).normalize(imagenet_stats)
  return data

In [ ]:
learn50 = create_cnn(get_data(8, int(2048/8)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/")
learn50.save('res50_0')
learn50.save('res50_8')
learn50 = create_cnn(get_data(16, int(2048/16)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_8')
learn50.save('res50_16')
learn50 = create_cnn(get_data(24, int(2048/24)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_16')
learn50.save('res50_24')
learn50 = create_cnn(get_data(32, int(2048/32)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_24')
learn50.save('res50_32')
learn50 = create_cnn(get_data(64, int(2048/64)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_32')
learn50.save('res50_64')
learn50 = create_cnn(get_data(128, int(2048/128)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_64')
learn50.save('res50_128')
learn50 = create_cnn(get_data(224, int(2048/224)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_128')
learn50.save('res50_224')

In [ ]:
def train_model(sz, i):
  learn50 = cnn_learner(get_data(sz, int(2048/sz)), models.resnet50, metrics=[error_rate, accuracy], model_dir="/tmp/model/").load('res50_'+str(sz-8))
  learn50.fit_one_cycle(6*i)
  learn50.lr_find()
  learn50.recorder.plot()
  learn50.unfreeze()
  learn50.fit_one_cycle(2*i)
  learn50.save('res50_'+str(sz))

In [ ]:
from PIL import Image

In [ ]:
train.head(5),test.head(5)

In [ ]:
train_images=np.array(train.iloc[:,0])
train_images

In [ ]:
img_dir='/kaggle/input/he_challenge_data/data/train/'

In [ ]:
train_images=np.array(train.iloc[:,0])
imagearr=[]
for i in train_images:
    img=Image.open(img_dir+i).convert('L')
    img=img.resize((150,150),Image.ANTIALIAS)
    imagearr.append(np.array(img))

In [ ]:
imagearr[0]

In [ ]:
trainImages = np.array([i for i in imagearr]).reshape(-1, 150, 150, 1)
print(trainImages.shape)
plt.imshow(np.asarray(imagearr)[5])

In [ ]:
img_dir='/kaggle/input/he_challenge_data/data/test/'
test_images=np.array(test.iloc[:,0])
imagearr=[]
for i in test_images:
    img=Image.open(img_dir+i).convert('L')
    img=img.resize((150,150),Image.ANTIALIAS)
    imagearr.append(np.array(img))

In [ ]:
# np.asarray(imagearr)
testImages = np.array([i for i in imagearr]).reshape(-1, 150, 150, 1)
print(testImages.shape)

In [ ]:
trainLabels = train['label'].values
from keras.utils import np_utils
trainLabels.shape
trainLabels = np_utils.to_categorical(trainLabels)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (5, 5), activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(4,4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(103, activation = 'softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])

In [ ]:
model.fit(trainImages, trainLabels, batch_size = 100, epochs = 5, verbose = 1)

In [ ]:
predictions = model.predict_classes(testImages)
predictions

In [ ]:
predictions

In [ ]:

test['category']=predictions
test.head()

In [ ]:
cd /kaggle/working

In [ ]:
df = pd.DataFrame({'image_id':test['image_id'], 'label':predictions}, columns=['image', 'label'])
df.to_csv('submission1.csv', index=False)

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "c.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(filename = 'c.csv')

**final**

In [ ]:
path= '/kaggle/input/he_challenge_data/data/train/'
tfms = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False,xtra_tfms=[cutout()])
data = (ImageList.from_csv(path, csv_name = 'trainfinal.csv')
        .split_by_rand_pct()              
        .label_from_df()            
        .add_test_folder(test_folder = 'test')              
        .transform(tfms, size=400)
        .databunch(num_workers=0,bs=8))

In [ ]:
## to see the images in train with there labels
data.show_batch(rows=3, figsize=(8,10))

In [ ]:
## print the target classes
print(data.classes)

In [ ]:
## load the pretrained imagenet model
## you can try other models from this link
## https://docs.fast.ai/vision.models.html
learn = cnn_learner(data, models.densenet161, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
from fastai import *
from fastai.vision import *
from sklearn.model_selection import StratifiedKFold
from pathlib import Path
import shutil
from sklearn.metrics import f1_score, confusion_matrix

np.random.seed(1786)
ROOT = "/tmp/data2"

In [ ]:
#cd /kaggle/input/he_challenge_data/data
def read_data(root):
    train_df = pd.read_csv('trainfinal.csv')
    test_df = pd.read_csv('testfinal.csv')
    return train_df, test_df

In [ ]:
class SaveBestModel(Recorder):
    def __init__(self, learn,name='best_model'):
        super().__init__(learn)
        self.name = name
        self.best_loss = None
        self.best_acc = None
        self.save_method = self.save_when_acc
        
    def save_when_acc(self, metrics):        
        loss, acc = metrics[0], metrics[1]
        if (self.best_acc is None) or (acc > self.best_acc) or (loss < self.best_loss):
            self.best_acc = acc
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Save the best acc {:.5f}".format(self.best_acc))
        elif acc == self.best_acc and  loss < self.best_loss:
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Acc is eq,Save the lower loss {:.5f}".format(self.best_loss))
            
    def on_epoch_end(self,last_metrics=MetricsList,**kwargs:Any):
        self.save_method(last_metrics)

In [ ]:
pip install fastai


In [ ]:
#Path(ROOT).mkdir(exist_ok=True, parents=True)
src1 = "/kaggle/input/he_challenge_data/data"
shutil.copytree(src1, ROOT)


if __name__=="__main__":
    train_df, test_df = read_data(ROOT)
    print(train_df.shape, test_df.shape)
    
    cvlist = list(StratifiedKFold(8, random_state=12345786).split(train_df, train_df.label))
    
    tfms1 = get_transforms(max_zoom=1.5)
    test_preds_all = []
    val_preds_all = []
    for i in range(2):
        print("Starting fold {}".format(i))
        tr_idx, val_idx = cvlist[i]
        print(tr_idx.shape, val_idx.shape)
        src = (ImageList.from_df(train_df, path=ROOT, folder="train").split_by_idxs(tr_idx, val_idx)
                                                            .label_from_df())
        data = ImageDataBunch.create_from_ll(src, ds_tfms=tfms1, size=400, bs=8, resize_method=3).normalize(imagenet_stats)
        data.add_test(ImageList.from_df(test_df, path=ROOT, folder="test"))
        learn = cnn_learner(data, models.densenet169, metrics=accuracy, ps=0.5)
        # learn.model[0].load_state_dict(torch.load("../input/save-body-weights-marvel/bestmodel_body.path"))
        cb = SaveBestModel(learn, name="bestmodel_{}".format(i))
        #learn.fit(1, callbacks=cb)
        learn.fit_one_cycle(6)
        learn.unfreeze()
        learn.fit_one_cycle(10, max_lr=1e-4, callbacks=[cb])
        #learn.fit_one_cycle(10, max_lr=5e-5, callbacks=[cb])
        #learn.fit_one_cycle(10, max_lr=1e-5, callbacks=[cb])
        #learn.fit_one_cycle(10, max_lr=5e-6, callbacks=[cb])
        learn.fit_one_cycle(5, max_lr=1e-6, callbacks=[cb])
        learn.load("bestmodel_{}".format(i))
        val_preds, y = learn.TTA(ds_type=DatasetType.Valid)
        val_preds = np.exp(val_preds.numpy())
        print("F1 score for this fold ",f1_score(y.numpy(), np.argmax(val_preds,axis=1), average='weighted'))
        test_preds = np.exp(learn.TTA(ds_type=DatasetType.Test)[0].numpy())
        test_preds_all.append(test_preds)
        val_preds_all.append(val_preds)
        fname = "bestmodel_{}.pth".format(i)
        src = str(Path(ROOT) / "models" / fname)
        shutil.copy(src, fname)
    test_preds_all = np.mean(test_preds_all, axis=0)
    val_preds_all = np.concatenate(val_preds_all, axis=0)

    np.save("test_preds.npy", test_preds_all)
    np.save("val_preds.npy", val_preds_all)
    sub = test_df[["image"]]
    sub["category"] = np.argmax(test_preds_all, axis=1)
    sub.to_csv("subv1.csv", index=False)

In [ ]:

test_preds = np.exp(learn.TTA(ds_type=DatasetType.Test)[0].numpy())

In [ ]:
test_preds_all = np.mean(test_preds_all, axis=0)
val_preds_all = np.concatenate(val_preds_all, axis=0)

In [ ]:
preds = np.argmax(test_preds_all, 1)
preds

In [ ]:
import numpy as np
import pandas as pd
a = np.array(preds)
print(a)
a=a+1
print(a.max)
print(data.test_ds.x[0])

In [ ]:
a

In [ ]:
cd /kaggle/working

In [ ]:
df = pd.DataFrame({'label':a}, columns=['label'])
df.to_csv('gamerr.csv', index=False)

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "gamerr.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(filename = 'gamerr.csv')